Parameter Sensitivity Analysis (Grid Search)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

def calculate_features(df):
    """Engineers all necessary features from the raw OHLCV data."""
    df = df.copy()
    
    # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
    delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    
    # Handle division by zero and NaN values
    rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # 2. Bollinger Bands
    df['sma_20'] = df['spy_close'].rolling(window=20).mean()
    df['std_20'] = df['spy_close'].rolling(window=20).std()
    df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
    df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

    # 3. Rolling Z-Score
    df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

    # 4. Average True Range (ATR) - Fixed column references
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
    low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr'] = tr.rolling(window=14).mean()
    
    return df

def get_market_data(start_date="2005-01-01"):
    """Downloads SPY and VIX data and engineers features."""
    spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
    vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

    # Reset any potential multi-level indexes and ensure clean column structure
    if isinstance(spy_data.columns, pd.MultiIndex):
        spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
    if isinstance(vix_data.columns, pd.MultiIndex):
        vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
    
    # Select only the columns we need
    spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
    vix_data = vix_data[['Close']].copy()
    vix_data = vix_data.rename(columns={'Close': 'vix'})
    
    # Align the date indices properly
    common_dates = spy_data.index.intersection(vix_data.index)
    spy_data = spy_data.loc[common_dates]
    vix_data = vix_data.loc[common_dates]
    
    # Combine into a single DataFrame using merge
    df = spy_data.copy()
    df['vix'] = vix_data['vix']
    df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

    # Calculate features using the function
    df = calculate_features(df)
    
    # Calculate daily returns for the backtest
    df['daily_return'] = df['spy_close'].pct_change()

    return df.dropna()

def generate_vol_norm_rsi_signals(df, rsi_entry=30, rsi_exit=50, atr_multiplier=1.0, vix_threshold=40):
    """
    Generates trading signals based on the Volatility-Normalised RSI strategy.
    Returns a DataFrame with a 'signal' column (1 for long, -1 for short, 0 for flat).
    """
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0

    # Entry Conditions
    long_entry_condition = (df['rsi'] < rsi_entry) & \
                           (df['spy_close'] < (df['spy_close'].shift(1) - (atr_multiplier * df['atr'])))
    
    short_entry_condition = (df['rsi'] > (100 - rsi_entry)) & \
                            (df['spy_close'] > (df['spy_close'].shift(1) + (atr_multiplier * df['atr'])))

    # Exit Conditions
    long_exit_condition = df['rsi'] > rsi_exit
    short_exit_condition = df['rsi'] < (100 - rsi_exit)

    # State machine to hold positions
    position = 0
    for i in range(len(df)):
        if position == 0: # If flat
            if long_entry_condition.iloc[i]:
                position = 1
            elif short_entry_condition.iloc[i]:
                position = -1
        elif position == 1: # If long
            if long_exit_condition.iloc[i]:
                position = 0
        elif position == -1: # If short
            if short_exit_condition.iloc[i]:
                position = 0
        signals['signal'].iloc[i] = position

    # VIX Regime Filter - Fixed the assignment
    vix_high_mask = df['vix'] > vix_threshold
    signals.loc[vix_high_mask, 'signal'] = 0
    
    return signals['signal']

def run_vectorised_backtest(data, signals, spread=0.0001, commission=0.0035, slippage_pct=0.0001):
    """
    Runs a vectorised backtest with realistic transaction costs.
    
    Args:
        data (pd.DataFrame): DataFrame with 'daily_return' and other market data.
        signals (pd.Series): Series with trading signals (1, -1, 0).
        spread (float): Bid-ask spread as a fraction (e.g., 0.01% = 0.0001).
        commission (float): Per-trade commission as a percentage of position value.
        slippage_pct (float): Slippage as a percentage of position value.
    
    Returns:
        pd.Series: Daily returns of the strategy.
    """
    # Lag signals to prevent lookahead bias (trade on next day's open)
    positions = signals.shift(1).fillna(0)
    
    # Calculate strategy returns without costs
    strategy_returns = positions * data['daily_return']
    
    # Identify trade days (when position changes)
    trades = positions.diff().abs()
    
    # Calculate transaction costs
    # 1. Spread cost (incurred on every trade) - simplified as half the spread per trade
    spread_cost = trades * (spread / 2)
    
    # 2. Commission cost (as a percentage of the trade value)
    commission_cost = trades * commission
    
    # 3. Slippage cost (as a percentage of the trade value)
    slippage_cost = trades * slippage_pct
    
    total_costs = spread_cost + commission_cost + slippage_cost
    
    # Subtract costs from returns on trade days
    # Only apply costs when trades actually occur
    strategy_returns = strategy_returns - total_costs
    
    return strategy_returns.fillna(0)

def calculate_sharpe_ratio(returns):
    """Calculate Sharpe ratio."""
    if returns.std() == 0:
        return np.nan
    return returns.mean() / returns.std() * np.sqrt(252)  # Annualized

def display_performance(strategy_returns):
    """Prints a performance summary."""
    print("--- Strategy Performance ---")
    print(f"Total Return: {(strategy_returns + 1).prod() - 1:.4f}")
    print(f"Annual Return: {strategy_returns.mean() * 252:.4f}")
    print(f"Annual Volatility: {strategy_returns.std() * np.sqrt(252):.4f}")
    print(f"Sharpe Ratio: {calculate_sharpe_ratio(strategy_returns):.4f}")
    print(f"Max Drawdown: {calculate_max_drawdown(strategy_returns):.4f}")
    print(f"Win Rate: {calculate_win_rate(strategy_returns):.4f}")
    
def calculate_max_drawdown(returns):
    """Calculate maximum drawdown."""
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown.min()

def calculate_win_rate(returns):
    """Calculate win rate."""
    non_zero_returns = returns[returns != 0]
    if len(non_zero_returns) == 0:
        return 0.0
    return (non_zero_returns > 0).sum() / len(non_zero_returns)

# 1. Load data and define in-sample period
full_data = get_market_data()
in_sample_data = full_data.loc['2005-01-01':'2014-12-31']

# 2. Define parameter grid
rsi_thresholds = [20, 25, 30, 35]
atr_multipliers = [0.5, 1.0, 1.5, 2.0]

results = []

# 3. Run grid search
print("Running parameter grid search...")
for rsi in rsi_thresholds:
    for atr in atr_multipliers:
        signals = generate_vol_norm_rsi_signals(
            in_sample_data, 
            rsi_entry=rsi, 
            atr_multiplier=atr
        )
        
        returns = run_vectorised_backtest(in_sample_data, signals)
        sharpe = calculate_sharpe_ratio(returns)
        
        results.append({
            'rsi_threshold': rsi,
            'atr_multiplier': atr,
            'sharpe_ratio': sharpe
        })
        print(f"RSI: {rsi}, ATR: {atr} -> Sharpe: {sharpe:.2f}")

# 4. Process and visualize results
results_df = pd.DataFrame(results)
heatmap_data = results_df.pivot(
    index='atr_multiplier', 
    columns='rsi_threshold', 
    values='sharpe_ratio'
)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, cmap="YlGnBu", fmt=".2f", cbar_kws={'label': 'Sharpe Ratio'})
plt.title("Parameter Sensitivity Heatmap: In-Sample Sharpe Ratio")
plt.xlabel("RSI Entry Threshold")
plt.ylabel("ATR Multiplier")
plt.show()

# Identify the best parameters from the "plateau"
# Find the best combination
best_idx = results_df['sharpe_ratio'].idxmax()
best_params = {
    'rsi_entry': results_df.loc[best_idx, 'rsi_threshold'],
    'atr_multiplier': results_df.loc[best_idx, 'atr_multiplier']
}
print(f"\nBest parameters: {best_params}")
print(f"Best Sharpe ratio: {results_df.loc[best_idx, 'sharpe_ratio']:.4f}")

# Define out-of-sample period
out_of_sample_data = full_data.loc['2015-01-01':]

# Generate signals and run backtest on OOS data with best parameters
oos_signals = generate_vol_norm_rsi_signals(out_of_sample_data, **best_params)
oos_returns = run_vectorised_backtest(out_of_sample_data, oos_signals)

print("\n--- Out-of-Sample Performance ---")
display_performance(oos_returns)

# Plot out-of-sample performance
plt.figure(figsize=(15, 7))
cumulative_returns = (1 + oos_returns).cumprod()
plt.plot(cumulative_returns.index, cumulative_returns, label='Out-of-Sample Strategy Returns')
plt.title('Out-of-Sample Strategy Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

Running parameter grid search...
RSI: 20, ATR: 0.5 -> Sharpe: -0.58
RSI: 20, ATR: 1.0 -> Sharpe: -0.49
RSI: 20, ATR: 1.5 -> Sharpe: -0.35


C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  signals['signal'].iloc[i] = position
C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: Chained

RSI: 20, ATR: 2.0 -> Sharpe: -0.33
RSI: 25, ATR: 0.5 -> Sharpe: -0.63


C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  signals['signal'].iloc[i] = position
C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: Chained

...

C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  signals['signal'].iloc[i] = position
C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: Chained

RSI: 35, ATR: 0.5 -> Sharpe: -0.64
RSI: 35, ATR: 1.0 -> Sharpe: -0.38


C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  signals['signal'].iloc[i] = position
C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: Chained

RSI: 35, ATR: 1.5 -> Sharpe: -0.58


C:\Users\USER\AppData\Local\Temp\ipykernel_21968\2910427695.py:105: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  signals['signal'].iloc[i] = position


# Price Target Exit (long only)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

def calculate_features(df):
    """Engineers all necessary features from the raw OHLCV data."""
    df = df.copy()
    
    # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
    delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    
    # Handle division by zero and NaN values
    rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # 2. Bollinger Bands
    df['sma_20'] = df['spy_close'].rolling(window=20).mean()
    df['std_20'] = df['spy_close'].rolling(window=20).std()
    df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
    df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

    # 3. Rolling Z-Score
    df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

    # 4. Average True Range (ATR) - Fixed column references
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
    low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr'] = tr.rolling(window=14).mean()

    # 5. Add Volume Moving Average
    df['volume_sma_50'] = df['Volume'].rolling(window=50).mean()
    
    return df

def get_market_data(start_date="2005-01-01"):
    """Downloads SPY and VIX data and engineers features."""
    spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
    vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

    # Reset any potential multi-level indexes and ensure clean column structure
    if isinstance(spy_data.columns, pd.MultiIndex):
        spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
    if isinstance(vix_data.columns, pd.MultiIndex):
        vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
    
    # Select only the columns we need
    spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
    vix_data = vix_data[['Close']].copy()
    vix_data = vix_data.rename(columns={'Close': 'vix'})
    
    # Align the date indices properly
    common_dates = spy_data.index.intersection(vix_data.index)
    spy_data = spy_data.loc[common_dates]
    vix_data = vix_data.loc[common_dates]
    
    # Combine into a single DataFrame using merge
    df = spy_data.copy()
    df['vix'] = vix_data['vix']
    df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

    # Calculate features using the function
    df = calculate_features(df)
    
    # Calculate daily returns for the backtest
    df['daily_return'] = df['spy_close'].pct_change()

    return df.dropna()

def generate_sma_exit_signals(df, rsi_entry=30, atr_multiplier=1.5, vix_threshold=40):
    """
    Generates signals with an exit condition based on crossing the 20-day SMA.
    """
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0.0
    position = 0  # 0: flat, 1: long, -1: short

    # --- ENTRY CONDITIONS (Modified) ---
    long_entry = (df['rsi'] < rsi_entry) & \
                 (df['spy_close'] < (df['spy_close'].shift(1) - (atr_multiplier * df['atr']))) & \
                 (df['spy_close'] < df['bollinger_lower'])  # <-- ADD THIS LINE
    
    short_entry = (df['rsi'] > (100 - rsi_entry)) & \
                  (df['spy_close'] > (df['spy_close'].shift(1) + (atr_multiplier * df['atr']))) & \
                  (df['spy_close'] > df['bollinger_upper']) # <-- ADD THIS LINE

    for i in range(1, len(df)):
        # --- EXIT CONDITIONS ---
        # Exit long if price crosses above the 20-day SMA
        if position == 1 and (df['spy_close'].iloc[i] > df['sma_20'].iloc[i]):
            position = 0
        # Exit short if price crosses below the 20-day SMA
        elif position == -1 and (df['spy_close'].iloc[i] < df['sma_20'].iloc[i]):
            position = 0
        
        # --- ENTRY LOGIC ---
        if position == 0:
            if long_entry.iloc[i]:
                position = 1
            elif short_entry.iloc[i]:
                position = 0 # stop shorting
        
        signals['signal'].iloc[i] = position

    # VIX Regime Filter
    signals.loc[df['vix'] > vix_threshold, 'signal'] = 0
    
    return signals['signal']

def run_vectorised_backtest(data, signals, spread=0.0001, commission=0.0035, slippage_pct=0.0001):
    """
    Runs a vectorised backtest with realistic transaction costs.
    
    Args:
        data (pd.DataFrame): DataFrame with 'daily_return' and other market data.
        signals (pd.Series): Series with trading signals (1, -1, 0).
        spread (float): Bid-ask spread as a fraction (e.g., 0.01% = 0.0001).
        commission (float): Per-trade commission as a percentage of position value.
        slippage_pct (float): Slippage as a percentage of position value.
    
    Returns:
        pd.Series: Daily returns of the strategy.
    """
    # Lag signals to prevent lookahead bias (trade on next day's open)
    positions = signals.shift(1).fillna(0)
    
    # Calculate strategy returns without costs
    strategy_returns = positions * data['daily_return']
    
    # Identify trade days (when position changes)
    trades = positions.diff().abs()
    
    # Calculate transaction costs
    # 1. Spread cost (incurred on every trade) - simplified as half the spread per trade
    spread_cost = trades * (spread / 2)
    
    # 2. Commission cost (as a percentage of the trade value)
    commission_cost = trades * commission
    
    # 3. Slippage cost (as a percentage of the trade value)
    slippage_cost = trades * slippage_pct
    
    total_costs = spread_cost + commission_cost + slippage_cost
    
    # Subtract costs from returns on trade days
    # Only apply costs when trades actually occur
    strategy_returns = strategy_returns - total_costs
    
    return strategy_returns.fillna(0)

def calculate_sharpe_ratio(returns):
    """Calculate Sharpe ratio."""
    if returns.std() == 0:
        return np.nan
    return returns.mean() / returns.std() * np.sqrt(252)  # Annualised

def display_performance(strategy_returns):
    """Prints a performance summary."""
    print("--- Strategy Performance ---")
    print(f"Total Return: {(strategy_returns + 1).prod() - 1:.4f}")
    print(f"Annual Return: {strategy_returns.mean() * 252:.4f}")
    print(f"Annual Volatility: {strategy_returns.std() * np.sqrt(252):.4f}")
    print(f"Sharpe Ratio: {calculate_sharpe_ratio(strategy_returns):.4f}")
    print(f"Max Drawdown: {calculate_max_drawdown(strategy_returns):.4f}")
    print(f"Win Rate: {calculate_win_rate(strategy_returns):.4f}")
    
def calculate_max_drawdown(returns):
    """Calculate maximum drawdown."""
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown.min()

def calculate_win_rate(returns):
    """Calculate win rate."""
    non_zero_returns = returns[returns != 0]
    if len(non_zero_returns) == 0:
        return 0.0
    return (non_zero_returns > 0).sum() / len(non_zero_returns)

# 1. Load data and define in-sample period
full_data = get_market_data()
in_sample_data = full_data.loc['2005-01-01':'2014-12-31']

# 2. Define parameter grid
rsi_thresholds = [20, 25, 30, 35]
atr_multipliers = [0.5, 1.0, 1.5, 2.0]

results = []

# 3. Run grid search
print("Running parameter grid search...")
for rsi in rsi_thresholds:
    for atr in atr_multipliers:
        signals = generate_vol_norm_rsi_signals(
            in_sample_data, 
            rsi_entry=rsi, 
            atr_multiplier=atr
        )
        
        returns = run_vectorised_backtest(in_sample_data, signals)
        sharpe = calculate_sharpe_ratio(returns)
        
        results.append({
            'rsi_threshold': rsi,
            'atr_multiplier': atr,
            'sharpe_ratio': sharpe
        })
        print(f"RSI: {rsi}, ATR: {atr} -> Sharpe: {sharpe:.2f}")

# 4. Process and visualize results
results_df = pd.DataFrame(results)
heatmap_data = results_df.pivot(
    index='atr_multiplier', 
    columns='rsi_threshold', 
    values='sharpe_ratio'
)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, cmap="YlGnBu", fmt=".2f", cbar_kws={'label': 'Sharpe Ratio'})
plt.title("Parameter Sensitivity Heatmap: In-Sample Sharpe Ratio")
plt.xlabel("RSI Entry Threshold")
plt.ylabel("ATR Multiplier")
plt.show()

# Identify the best parameters from the "plateau"
# Find the best combination
best_idx = results_df['sharpe_ratio'].idxmax()
best_params = {
    'rsi_entry': results_df.loc[best_idx, 'rsi_threshold'],
    'atr_multiplier': results_df.loc[best_idx, 'atr_multiplier']
}
print(f"\nBest parameters: {best_params}")
print(f"Best Sharpe ratio: {results_df.loc[best_idx, 'sharpe_ratio']:.4f}")

# Define out-of-sample period
out_of_sample_data = full_data.loc['2015-01-01':]

# Generate signals and run backtest on OOS data with best parameters
oos_signals = generate_sma_exit_signals(out_of_sample_data, **best_params)
oos_returns = run_vectorised_backtest(out_of_sample_data, oos_signals)

print("\n--- Out-of-Sample Performance ---")
display_performance(oos_returns)

# Plot out-of-sample performance
plt.figure(figsize=(15, 7))
cumulative_returns = (1 + oos_returns).cumprod()
plt.plot(cumulative_returns.index, cumulative_returns, label='Out-of-Sample Strategy Returns')
plt.title('Out-of-Sample Strategy Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

# Bollinger + Volume Filter 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

def calculate_features(df):
    """Engineers all necessary features from the raw OHLCV data."""
    df = df.copy()
    
    # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
    delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    
    # Handle division by zero and NaN values
    rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # 2. Bollinger Bands
    df['sma_20'] = df['spy_close'].rolling(window=20).mean()
    df['std_20'] = df['spy_close'].rolling(window=20).std()
    df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
    df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

    # 3. Rolling Z-Score
    df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

    # 4. Average True Range (ATR) - Fixed column references
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
    low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr'] = tr.rolling(window=14).mean()

    # 5. Add Volume Moving Average
    df['volume_sma_50'] = df['Volume'].rolling(window=50).mean()
    
    return df

def get_market_data(start_date="2005-01-01"):
    """Downloads SPY and VIX data and engineers features."""
    spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
    vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

    # Reset any potential multi-level indexes and ensure clean column structure
    if isinstance(spy_data.columns, pd.MultiIndex):
        spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
    if isinstance(vix_data.columns, pd.MultiIndex):
        vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
    
    # Select only the columns we need
    spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
    vix_data = vix_data[['Close']].copy()
    vix_data = vix_data.rename(columns={'Close': 'vix'})
    
    # Align the date indices properly
    common_dates = spy_data.index.intersection(vix_data.index)
    spy_data = spy_data.loc[common_dates]
    vix_data = vix_data.loc[common_dates]
    
    # Combine into a single DataFrame using merge
    df = spy_data.copy()
    df['vix'] = vix_data['vix']
    df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

    # Calculate features using the function
    df = calculate_features(df)
    
    # Calculate daily returns for the backtest
    df['daily_return'] = df['spy_close'].pct_change()

    return df.dropna()

def generate_sma_exit_signals(df, rsi_entry=30, atr_multiplier=1.5, vix_threshold=40):
    """
    Generates signals with an exit condition based on crossing the 20-day SMA.
    """
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0.0
    position = 0  # 0: flat, 1: long, -1: short

    # --- ENTRY CONDITIONS (Modified with Volume) ---
    long_entry = (df['rsi'] < rsi_entry) & \
                 (df['spy_close'] < df['bollinger_lower']) & \
                 (df['Volume'] > df['volume_sma_50'] * 1.5) 
    
    short_entry = (df['rsi'] > (100 - rsi_entry)) & \
                  (df['spy_close'] > df['bollinger_upper']) & \
                  (df['Volume'] > df['volume_sma_50'] * 1.5)

    for i in range(1, len(df)):
        # --- EXIT CONDITIONS ---
        # Exit long if price crosses above the 20-day SMA
        if position == 1 and (df['spy_close'].iloc[i] > df['sma_20'].iloc[i]):
            position = 0
        # Exit short if price crosses below the 20-day SMA
        elif position == -1 and (df['spy_close'].iloc[i] < df['sma_20'].iloc[i]):
            position = 0
        
        # --- ENTRY LOGIC ---
        if position == 0:
            if long_entry.iloc[i]:
                position = 1
            elif short_entry.iloc[i]:
                position = 0 # stop shorting
        
        signals['signal'].iloc[i] = position

    # VIX Regime Filter
    signals.loc[df['vix'] > vix_threshold, 'signal'] = 0
    
    return signals['signal']

def run_vectorised_backtest(data, signals, spread=0.0001, commission=0.0035, slippage_pct=0.0001):
    """
    Runs a vectorised backtest with realistic transaction costs.
    
    Args:
        data (pd.DataFrame): DataFrame with 'daily_return' and other market data.
        signals (pd.Series): Series with trading signals (1, -1, 0).
        spread (float): Bid-ask spread as a fraction (e.g., 0.01% = 0.0001).
        commission (float): Per-trade commission as a percentage of position value.
        slippage_pct (float): Slippage as a percentage of position value.
    
    Returns:
        pd.Series: Daily returns of the strategy.
    """
    # Lag signals to prevent lookahead bias (trade on next day's open)
    positions = signals.shift(1).fillna(0)
    
    # Calculate strategy returns without costs
    strategy_returns = positions * data['daily_return']
    
    # Identify trade days (when position changes)
    trades = positions.diff().abs()
    
    # Calculate transaction costs
    # 1. Spread cost (incurred on every trade) - simplified as half the spread per trade
    spread_cost = trades * (spread / 2)
    
    # 2. Commission cost (as a percentage of the trade value)
    commission_cost = trades * commission
    
    # 3. Slippage cost (as a percentage of the trade value)
    slippage_cost = trades * slippage_pct
    
    total_costs = spread_cost + commission_cost + slippage_cost
    
    # Subtract costs from returns on trade days
    # Only apply costs when trades actually occur
    strategy_returns = strategy_returns - total_costs
    
    return strategy_returns.fillna(0)

def calculate_sharpe_ratio(returns):
    """Calculate Sharpe ratio."""
    if returns.std() == 0:
        return np.nan
    return returns.mean() / returns.std() * np.sqrt(252)  # Annualised

def display_performance(strategy_returns):
    """Prints a performance summary."""
    print("--- Strategy Performance ---")
    print(f"Total Return: {(strategy_returns + 1).prod() - 1:.4f}")
    print(f"Annual Return: {strategy_returns.mean() * 252:.4f}")
    print(f"Annual Volatility: {strategy_returns.std() * np.sqrt(252):.4f}")
    print(f"Sharpe Ratio: {calculate_sharpe_ratio(strategy_returns):.4f}")
    print(f"Max Drawdown: {calculate_max_drawdown(strategy_returns):.4f}")
    print(f"Win Rate: {calculate_win_rate(strategy_returns):.4f}")
    
def calculate_max_drawdown(returns):
    """Calculate maximum drawdown."""
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown.min()

def calculate_win_rate(returns):
    """Calculate win rate."""
    non_zero_returns = returns[returns != 0]
    if len(non_zero_returns) == 0:
        return 0.0
    return (non_zero_returns > 0).sum() / len(non_zero_returns)

# 1. Load data and define in-sample period
full_data = get_market_data()
in_sample_data = full_data.loc['2005-01-01':'2014-12-31']

# 2. Define parameter grid
rsi_thresholds = [20, 25, 30, 35]
atr_multipliers = [0.5, 1.0, 1.5, 2.0]

results = []

# 3. Run grid search
print("Running parameter grid search...")
for rsi in rsi_thresholds:
    for atr in atr_multipliers:
        signals = generate_vol_norm_rsi_signals(
            in_sample_data, 
            rsi_entry=rsi, 
            atr_multiplier=atr
        )
        
        returns = run_vectorised_backtest(in_sample_data, signals)
        sharpe = calculate_sharpe_ratio(returns)
        
        results.append({
            'rsi_threshold': rsi,
            'atr_multiplier': atr,
            'sharpe_ratio': sharpe
        })
        print(f"RSI: {rsi}, ATR: {atr} -> Sharpe: {sharpe:.2f}")

# 4. Process and visualize results
results_df = pd.DataFrame(results)
heatmap_data = results_df.pivot(
    index='atr_multiplier', 
    columns='rsi_threshold', 
    values='sharpe_ratio'
)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, cmap="YlGnBu", fmt=".2f", cbar_kws={'label': 'Sharpe Ratio'})
plt.title("Parameter Sensitivity Heatmap: In-Sample Sharpe Ratio")
plt.xlabel("RSI Entry Threshold")
plt.ylabel("ATR Multiplier")
plt.show()

# Identify the best parameters from the "plateau"
# Find the best combination
best_idx = results_df['sharpe_ratio'].idxmax()
best_params = {
    'rsi_entry': results_df.loc[best_idx, 'rsi_threshold'],
    'atr_multiplier': results_df.loc[best_idx, 'atr_multiplier']
}
print(f"\nBest parameters: {best_params}")
print(f"Best Sharpe ratio: {results_df.loc[best_idx, 'sharpe_ratio']:.4f}")

# Define out-of-sample period
out_of_sample_data = full_data.loc['2015-01-01':]

# Generate signals and run backtest on OOS data with best parameters
oos_signals = generate_sma_exit_signals(out_of_sample_data, **best_params)
oos_returns = run_vectorised_backtest(out_of_sample_data, oos_signals)

print("\n--- Out-of-Sample Performance ---")
display_performance(oos_returns)

# Plot out-of-sample performance
plt.figure(figsize=(15, 7))
cumulative_returns = (1 + oos_returns).cumprod()
plt.plot(cumulative_returns.index, cumulative_returns, label='Out-of-Sample Strategy Returns')
plt.title('Out-of-Sample Strategy Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

# Adjusting rsi_entry and atr_multiplier

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

def calculate_features(df):
    """Engineers all necessary features from the raw OHLCV data."""
    df = df.copy()
    
    # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
    delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    
    # Handle division by zero and NaN values
    rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # 2. Bollinger Bands
    df['sma_20'] = df['spy_close'].rolling(window=20).mean()
    df['std_20'] = df['spy_close'].rolling(window=20).std()
    df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
    df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

    # 3. Rolling Z-Score
    df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

    # 4. Average True Range (ATR) - Fixed column references
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
    low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr'] = tr.rolling(window=14).mean()

    # 5. Add Volume Moving Average
    df['volume_sma_50'] = df['Volume'].rolling(window=50).mean()
    
    return df

def get_market_data(start_date="2005-01-01"):
    """Downloads SPY and VIX data and engineers features."""
    spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
    vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

    # Reset any potential multi-level indexes and ensure clean column structure
    if isinstance(spy_data.columns, pd.MultiIndex):
        spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
    if isinstance(vix_data.columns, pd.MultiIndex):
        vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
    
    # Select only the columns we need
    spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
    vix_data = vix_data[['Close']].copy()
    vix_data = vix_data.rename(columns={'Close': 'vix'})
    
    # Align the date indices properly
    common_dates = spy_data.index.intersection(vix_data.index)
    spy_data = spy_data.loc[common_dates]
    vix_data = vix_data.loc[common_dates]
    
    # Combine into a single DataFrame using merge
    df = spy_data.copy()
    df['vix'] = vix_data['vix']
    df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

    # Calculate features using the function
    df = calculate_features(df)
    
    # Calculate daily returns for the backtest
    df['daily_return'] = df['spy_close'].pct_change()

    return df.dropna()

def generate_sma_exit_signals(df, rsi_entry=25, atr_multiplier=1.5, vix_threshold=40):

    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0.0
    position = 0  # 0: flat, 1: long, -1: short

    # --- ENTRY CONDITIONS (Modified with Volume) ---
    long_entry = (df['rsi'] < rsi_entry) & \
                 (df['spy_close'] < df['bollinger_lower']) & \
                 (df['Volume'] > df['volume_sma_50'] * 1.5) 
    
    short_entry = (df['rsi'] > (100 - rsi_entry)) & \
                  (df['spy_close'] > df['bollinger_upper']) & \
                  (df['Volume'] > df['volume_sma_50'] * 1.5)

    for i in range(1, len(df)):
        # --- EXIT CONDITIONS ---
        # Exit long if price crosses above the 20-day SMA
        if position == 1 and (df['spy_close'].iloc[i] > df['sma_20'].iloc[i]):
            position = 0
        # Exit short if price crosses below the 20-day SMA
        elif position == -1 and (df['spy_close'].iloc[i] < df['sma_20'].iloc[i]):
            position = 0
        
        # --- ENTRY LOGIC ---
        if position == 0:
            if long_entry.iloc[i]:
                position = 1
            elif short_entry.iloc[i]:
                position = 0 # stop shorting
        
        signals['signal'].iloc[i] = position

    # VIX Regime Filter
    signals.loc[df['vix'] > vix_threshold, 'signal'] = 0
    
    return signals['signal']

def run_vectorised_backtest(data, signals, spread=0.0001, commission=0.0035, slippage_pct=0.0001):
    """
    Runs a vectorised backtest with realistic transaction costs.
    
    Args:
        data (pd.DataFrame): DataFrame with 'daily_return' and other market data.
        signals (pd.Series): Series with trading signals (1, -1, 0).
        spread (float): Bid-ask spread as a fraction (e.g., 0.01% = 0.0001).
        commission (float): Per-trade commission as a percentage of position value.
        slippage_pct (float): Slippage as a percentage of position value.
    
    Returns:
        pd.Series: Daily returns of the strategy.
    """
    # Lag signals to prevent lookahead bias (trade on next day's open)
    positions = signals.shift(1).fillna(0)
    
    # Calculate strategy returns without costs
    strategy_returns = positions * data['daily_return']
    
    # Identify trade days (when position changes)
    trades = positions.diff().abs()
    
    # Calculate transaction costs
    # 1. Spread cost (incurred on every trade) - simplified as half the spread per trade
    spread_cost = trades * (spread / 2)
    
    # 2. Commission cost (as a percentage of the trade value)
    commission_cost = trades * commission
    
    # 3. Slippage cost (as a percentage of the trade value)
    slippage_cost = trades * slippage_pct
    
    total_costs = spread_cost + commission_cost + slippage_cost
    
    # Subtract costs from returns on trade days
    # Only apply costs when trades actually occur
    strategy_returns = strategy_returns - total_costs
    
    return strategy_returns.fillna(0)

def calculate_sharpe_ratio(returns):
    """Calculate Sharpe ratio."""
    if returns.std() == 0:
        return np.nan
    return returns.mean() / returns.std() * np.sqrt(252)  # Annualised

def display_performance(strategy_returns):
    """Prints a performance summary."""
    print("--- Strategy Performance ---")
    print(f"Total Return: {(strategy_returns + 1).prod() - 1:.4f}")
    print(f"Annual Return: {strategy_returns.mean() * 252:.4f}")
    print(f"Annual Volatility: {strategy_returns.std() * np.sqrt(252):.4f}")
    print(f"Sharpe Ratio: {calculate_sharpe_ratio(strategy_returns):.4f}")
    print(f"Max Drawdown: {calculate_max_drawdown(strategy_returns):.4f}")
    print(f"Win Rate: {calculate_win_rate(strategy_returns):.4f}")
    
def calculate_max_drawdown(returns):
    """Calculate maximum drawdown."""
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown.min()

def calculate_win_rate(returns):
    """Calculate win rate."""
    non_zero_returns = returns[returns != 0]
    if len(non_zero_returns) == 0:
        return 0.0
    return (non_zero_returns > 0).sum() / len(non_zero_returns)

# 1. Load data and define in-sample period
full_data = get_market_data()
in_sample_data = full_data.loc['2005-01-01':'2014-12-31']

# 2. Define parameter grid
rsi_thresholds = [20, 25, 30, 35]
atr_multipliers = [0.5, 1.0, 1.5, 2.0]

results = []

# 3. Run grid search
print("Running parameter grid search...")
for rsi in rsi_thresholds:
    for atr in atr_multipliers:
        signals = generate_vol_norm_rsi_signals(
            in_sample_data, 
            rsi_entry=rsi, 
            atr_multiplier=atr
        )
        
        returns = run_vectorised_backtest(in_sample_data, signals)
        sharpe = calculate_sharpe_ratio(returns)
        
        results.append({
            'rsi_threshold': rsi,
            'atr_multiplier': atr,
            'sharpe_ratio': sharpe
        })
        print(f"RSI: {rsi}, ATR: {atr} -> Sharpe: {sharpe:.2f}")

# 4. Process and visualise results
results_df = pd.DataFrame(results)
heatmap_data = results_df.pivot(
    index='atr_multiplier', 
    columns='rsi_threshold', 
    values='sharpe_ratio'
)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, cmap="YlGnBu", fmt=".2f", cbar_kws={'label': 'Sharpe Ratio'})
plt.title("Parameter Sensitivity Heatmap: In-Sample Sharpe Ratio")
plt.xlabel("RSI Entry Threshold")
plt.ylabel("ATR Multiplier")
plt.show()

# Identify the best parameters from the "plateau"
# Find the best combination
best_idx = results_df['sharpe_ratio'].idxmax()
best_params = {
    'rsi_entry': results_df.loc[best_idx, 'rsi_threshold'],
    'atr_multiplier': results_df.loc[best_idx, 'atr_multiplier']
}
print(f"\nBest parameters: {best_params}")
print(f"Best Sharpe ratio: {results_df.loc[best_idx, 'sharpe_ratio']:.4f}")

# Define out-of-sample period
out_of_sample_data = full_data.loc['2015-01-01':]

# Generate signals and run backtest on OOS data with best parameters
oos_signals = generate_sma_exit_signals(out_of_sample_data, **best_params)
oos_returns = run_vectorised_backtest(out_of_sample_data, oos_signals)

print("\n--- Out-of-Sample Performance ---")
display_performance(oos_returns)

# Plot out-of-sample performance
plt.figure(figsize=(15, 7))
cumulative_returns = (1 + oos_returns).cumprod()
plt.plot(cumulative_returns.index, cumulative_returns, label='Out-of-Sample Strategy Returns')
plt.title('Out-of-Sample Strategy Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()